In [1]:
# Generic OpenAI-compatible endpoint using ChatOpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage,SystemMessage
import os
default_model_name = os.environ["OPENAI_MODEL"]
pg_connection = os.environ["PGVECTOR_CONNECTION_STRING"]
llm = ChatOpenAI(model=default_model_name)

In [3]:
from langchain_ollama import OllamaEmbeddings 
from langchain_postgres.vectorstores import PGVector
 
 
# embed each chunk and insert it into the vector store
embeddings_model = OllamaEmbeddings(model="nomic-embed-text")
# Coba dengan nama default
try:
    db = PGVector.from_existing_index(
        embedding=embeddings_model,
        collection_name="langchain",  # Coba default
        connection=pg_connection,
    )
    print("Berhasil konek dengan collection 'langchain'")
except Exception as e1:
    print(f"Collection 'langchain' tidak ditemukan: {e1}")
    
    # Coba dengan nama lain yang mungkin
    try:
        db = PGVector.from_existing_index(
            embedding=embeddings_model,
            collection_name="nomic-embed-text",  # Nama model
            connection=pg_connection,
        )
        print("Berhasil konek dengan collection 'nomic-embed-text'")
    except Exception as e2:
        print(f"Collection 'nomic-embed-text' tidak ditemukan: {e2}")

Berhasil konek dengan collection 'langchain'


In [4]:
db.similarity_search("Q3", k=4)


[Document(id='1677ab38-7604-4515-869a-ddf06cf5bae8', metadata={'source': '../../tesla.md'}, page_content='---\n\n PART III\n\nITEM 10\\. DIRECTORS, EXECUTIVE OFFICERS AND CORPORATE GOVERNANCE\n\nThe information required by this Item 10 of Form 10-K will be included in our 2023 Proxy Statement to be filed with the Securities and Exchange Commission in connection with the solicitation of proxies for our 2023 Annual Meeting of Stockholders and is incorporated herein by reference. The 2023 Proxy Statement will be filed with the Securities and Exchange Commission within 120 days after the end of the fiscal year to which this report relates. \n\nITEM 11\\. EXECUTIVE COMPENSATION\n\nThe information required by this Item 11 of Form 10-K will be included in our 2023 Proxy Statement and is incorporated herein by reference. \n\nITEM 12\\. SECURITY OWNERSHIP OF CERTAIN BENEFICIAL OWNERS AND MANAGEMENT AND RELATED STOCKHOLDER MATTERS\n\nThe information required by this Item 12 of Form 10-K will be 

In [25]:
# create retriever
# retriever = db.as_retriever(search_kwargs={"k": 2})

from langchain_core.prompts import ChatPromptTemplate

retriever = db.as_retriever()

prompt = ChatPromptTemplate.from_template("""Always answer in indonesian language.Answer the question based only on
    the following context:
{context}

Question: {question}
""")
chain = prompt | llm


question="Apa saja produk TESLA?"
docs = retriever.get_relevant_documents(question)
result=chain.invoke({
    "context":docs,
    "question":question
})
print(result.content)

Berdasarkan dokumen yang tersedia, produk-produk Tesla meliputi:

1. **Kendaraan listrik**:
   - Model S
   - Model X
   - Model 3
   - Model Y
   - Cybertruck
   - Tesla Semi
   - Tesla Roadster

2. **Sistem energi**:
   - Sistem pembangkit dan penyimpanan energi (energy generation and storage systems)

Selain itu, Tesla juga menawarkan layanan terkait produk-produknya, seperti penjualan kendaraan bekas (used vehicle sales) dan layanan sewa (lease).
